## Provide list of paths for edits and run trials -> (maybe analyze results and add to CSV)

In [7]:
# General imports
import torch
import numpy as np
import os, sys
import json
from tqdm import tqdm
from datetime import datetime
import pandas as pd

In [2]:
# Local imports
sys.path.insert(0, 'src')
from utils import read_json, read_lists, informal_log, list_to_dict
from utils.model_utils import prepare_device
from parse_config import ConfigParser
# from data_loader import data_loaders
import datasets.datasets as module_data
import model.model as module_arch
from utils.knn_utils import analyze_knn, combine_results

from edit_knn import main as edit

In [17]:
# Obtain timestamp
timestamp = datetime.now().strftime(r'%m%d_%H%M%S')

In [3]:
# Define constants, paths
config_path = 'configs/copies/cinic10_imagenet_segmentation_edit_trials.json'
class_list_path = 'metadata/cinic-10/class_names.txt'
key_image_id = 'dog-train-n02114712_211'
paths_dir = os.path.join('paths', 'edits', key_image_id)

value_image_paths = [
    os.path.join(paths_dir, 'felzenszwalb_gaussian_0.txt'),
    os.path.join(paths_dir, 'felzenszwalb_masked_0.txt'),
    os.path.join(paths_dir, 'felzenszwalb_masked_1.txt'),
    os.path.join(paths_dir, 'quickshift_masked_0.txt'),
    os.path.join(paths_dir, 'slic_masked_0.txt'),
    os.path.join(paths_dir, 'watershed_masked_1.txt')
]
n_trials = len(value_image_paths)

key_image_paths = [os.path.join(paths_dir, 'incorrect_image.txt') for i in range(n_trials)]

In [4]:
# Load config file
config_dict = read_json(config_path)
# Load class list and obtain target class idx
class_list = read_lists(class_list_path)
class_idx_dict = list_to_dict(class_list)
target_class_name = key_image_id.split('-')[0]
target_class_idx = class_idx_dict[target_class_name]
# Set K
K = config_dict['editor']['K']

# Obtain target class

# config = ConfigParser(config_dict)

device, device_ids = prepare_device(config_dict['n_gpu'])

In [37]:
# Load datasets
data_loader_args = dict(config_dict["data_loader"]["args"])
dataset_args = dict(config_dict["dataset_args"])

# Create validation data loader
val_image_paths = read_lists(config_dict['dataset_paths']['valid_images'])
val_labels = read_lists(config_dict['dataset_paths']['valid_labels'])
val_paths_data_loader = torch.utils.data.DataLoader(
    module_data.CINIC10Dataset(
        data_dir="",
        image_paths=val_image_paths,
        labels=val_labels,
        return_paths=True,
        **dataset_args
    ),
    **data_loader_args
)

# Create data loader for covariance matrix
covariance_image_paths = read_lists(config_dict['covariance_dataset']['images'])
covariance_labels = read_lists(config_dict['covariance_dataset']['labels'])

covariance_data_loader = torch.utils.data.DataLoader(
    module_data.CINIC10Dataset(
        data_dir="",
        image_paths=covariance_image_paths,
        labels=covariance_labels,
        **dataset_args
    ),
    **data_loader_args
)

### Create log and save paths

In [51]:
# create log path to store the paths to each trial
save_root = config_dict['trainer']['save_dir']
save_trials_path = os.path.join(save_root, config_dict['name'], timestamp, 'trial_paths.txt')
if os.path.exists(save_trials_path):
    # os.remove(save_trials_path)
    print("Path {} already exists. Aborting.".format(save_trials_path))
else:
    progress_report_path = os.path.join(save_root, config_dict['name'], timestamp, 'progress_report.txt')
    if os.path.exists(progress_report_path):
        os.remove(progress_report_path)
    print("Saving path to directories for each trial to {}".format(save_trials_path))
    print("Printing progress reports to {}".format(progress_report_path))

Saving path to directories for each trial to saved/edit/trials/CINIC10_ImageNet-VGG_16/0110_125216/trial_paths.txt
Printing progress reports to saved/edit/trials/CINIC10_ImageNet-VGG_16/0110_125216/progress_report.txt


### Ensure all paths for keys and values exist

In [52]:
non_existent_key_paths = []
non_existent_value_paths = []
for key_path, value_path in zip(key_image_paths, value_image_paths):
    if not os.path.exists(key_path):
        non_existent_key_paths.append(key_path)
    if not os.path.exists(value_path):
        non_existent_value_paths.append(value_path)

if len(non_existent_key_paths) > 0:
    raise ValueError("Following paths are non existent: {}".format(non_existent_key_paths))
    
if len(non_existent_value_paths) > 0:
    raise ValueError("Following paths are non existent: {}".format(non_existent_value_paths))

## Run edit for each modified image

In [53]:
for idx, (key_path, value_path) in enumerate(zip(key_image_paths, value_image_paths)):
    # Print Progress
    informal_log("Starting Trial {}/{}...".format(idx + 1, n_trials), progress_report_path)
    
    # Create run id 
    value_image_id = os.path.splitext(os.path.basename(value_path))[0]
    run_id = os.path.join(timestamp, key_image_id, value_image_id)
    informal_log("Current run_id: {}".format(run_id), progress_report_path)
    
    # Read config file as json and make updates to key and value paths
    config_dict = read_json(config_path)
    config_dict['editor'].update({
        'key_paths_file': key_path,
        'value_paths_file': value_path
    })
    
    # Create config object
    config = ConfigParser(config_dict, run_id=run_id)
    
    # Log the current trial path
    informal_log(os.path.dirname(config.save_dir), save_trials_path)
    
    informal_log("Calling edit()...", progress_report_path)
    
    edit(
        config=config,
        val_paths_data_loader=val_paths_data_loader,
        covariance_data_loader=covariance_data_loader)
    
    # Print progress
    informal_log("Finished trial {}/{}. Results saved to {}".format(idx + 1, n_trials, os.path.dirname(config.save_dir)),
                progress_report_path)


Starting Trial 1/1...
Current run_id: 0110_125216/dog-train-n02114712_211/watershed_masked_1
saved/edit/trials/CINIC10_ImageNet-VGG_16/0110_125216/dog-train-n02114712_211/watershed_masked_1
Calling edit()...
Created CIFAR10PretrainedModelEdit model with 33646666 trainable parameters
Restored weights from external_code/PyTorch_CIFAR10/cifar10_models/state_dicts/vgg16_bn.pt
Using passed in data loader for validation.
Key images: ['data/cinic-10-imagenet/train/dog/n02114712_211.png']
Value images: ['saved/segmentations/dog-train-n02114712_211/modified_images/watershed_masked_1.png']
Masks: None
Prepared data for editing
Performing pre-edit metric & KNN calculations on validation set.


100%|██████████████████████████████████████████| 274/274 [02:17<00:00,  2.00it/s]


Pre-edit metrics: {'TP': array([6142, 5382, 5203, 3842, 4270, 2990, 5659, 5136, 4820, 4679]), 'TN': array([60115, 61110, 59441, 59902, 60157, 61756, 61050, 61747, 61643,
       61202]), 'FPs': array([2885, 1890, 3559, 3098, 2843, 1244, 1950, 1253, 1357, 1798]), 'FNs': array([ 858, 1618, 1797, 3158, 2730, 4010, 1341, 1864, 2180, 2321]), 'accuracy': 0.6874714285714286, 'per_class_accuracy': array([0.94652857, 0.94988571, 0.92348571, 0.91062857, 0.92038571,
       0.92494286, 0.95298571, 0.95547143, 0.94947143, 0.94115714]), 'per_class_accuracy_mean': 0.9374942857142857, 'precision': array([0.68040323, 0.74009901, 0.5938142 , 0.55360231, 0.60030929,
       0.706188  , 0.74372454, 0.80388167, 0.78031407, 0.72240235]), 'precision_mean': 0.6924738665961387, 'recall': array([0.87742857, 0.76885714, 0.74328571, 0.54885714, 0.61      ,
       0.42714286, 0.80842857, 0.73371429, 0.68857143, 0.66842857]), 'recall_mean': 0.6874714285714286, 'f1': array([0.7664566 , 0.75420404, 0.66019541, 0.551219

tensor(0.0054, device='cuda:0', grad_fn=<DivBackward0>): 100%|█| 40000/40000 [00:


Loss (orig, final): 0.07313124090433121 0.005388882476836443
L2 norm of weight change: 0.25711220502853394
Evaluating edited model on test set...
Performing post-edit metric & KNN calculations on validation set.


100%|██████████████████████████████████████████| 274/274 [02:08<00:00,  2.14it/s]


Post-edit metrics: {'TP': array([6093, 5316, 5118, 4275, 4212, 3289, 5077, 5170, 4922, 4638]), 'TN': array([60331, 61259, 59770, 58679, 60186, 61103, 62168, 61684, 61529,
       61401]), 'FPs': array([2669, 1741, 3230, 4321, 2814, 1897,  832, 1316, 1471, 1599]), 'FNs': array([ 907, 1684, 1882, 2725, 2788, 3711, 1923, 1830, 2078, 2362]), 'accuracy': 0.6872857142857143, 'per_class_accuracy': array([0.94891429, 0.95107143, 0.92697143, 0.89934286, 0.91997143,
       0.91988571, 0.96064286, 0.95505714, 0.9493    , 0.94341429]), 'per_class_accuracy_mean': 0.9374571428571429, 'precision': array([0.69538918, 0.7532946 , 0.61308098, 0.49732434, 0.59948762,
       0.63420748, 0.85919783, 0.79710145, 0.76990458, 0.74362674]), 'precision_mean': 0.696261480499339, 'recall': array([0.87042857, 0.75942857, 0.73114286, 0.61071429, 0.60171429,
       0.46985714, 0.72528571, 0.73857143, 0.70314286, 0.66257143]), 'recall_mean': 0.6872857142857143, 'f1': array([0.77312524, 0.75634915, 0.66692729, 0.548217

## Analyze KNN

In [5]:
save_trials_path = 'saved/edit/trials/CINIC10_ImageNet-VGG_16/0110_120730/trial_paths.txt'
trial_dirs = read_lists(save_trials_path)
knn_analysis_filename = 'knn_analysis_results.pth'

In [6]:
try:
    if not os.path.exists(save_trials_path):
        print("Path {} does not exist".format(save_trials_path))
    else:
        print("Obtaining trial paths from {}".format(save_trials_path))
except:
    print("Need to define save_trials_path.")



Obtaining trial paths from saved/edit/trials/CINIC10_ImageNet-VGG_16/0110_120730/trial_paths.txt


In [ ]:
## Process KNN results for each
n_trials = len(trial_dirs)
n_log = n_trials // 10 + 1  # log every 10%
progress_report_path = os.path.join(os.path.dirname(save_trials_path), 'progress_report_analysis.txt')

informal_log("Starting KNN analysis...", progress_report_path)
for trial_idx, trial_dir in tqdm(enumerate(trial_dirs)):
    # if trial_idx % n_log == 0:
    informal_log("Processing {}/{} trials. Currently processing {}".format(
        trial_idx+1, n_trials, os.path.basename(trial_dir)), progress_report_path)
                     
    results_save_dir = os.path.join(trial_dir, 'models')
    analyze_knn(
        restore_dir=results_save_dir,
        pre_edit_knn_path=os.path.join(results_save_dir, 'pre_edit_{}-nn.pth'.format(K)),
        post_edit_knn_path=os.path.join(results_save_dir, 'post_edit_{}-nn.pth'.format(K)),
        knn_analysis_filename=knn_analysis_filename,
        target_class_idx=target_class_idx,
        class_list=class_list,
        progress_report_path=progress_report_path,
        save_images=False,
        save_plots=True)
    


## Convert to CSV for all trials

In [8]:
data = []
# Iterate through all trials
for trial_idx, trial_dir in tqdm(enumerate(trial_dirs)):
    # Obtain key ID from path
    key_id = os.path.basename(os.path.dirname(trial_dir))
    id_class = key_id.split('-')[0]
    if id_class not in class_list:
        raise ValueError("Invalid key_id {}".format(key_id))
    # Obtain value ID from path
    val_id = os.path.basename(trial_dir)
    # Join to make a data ID
    data_id = os.path.join(key_id, val_id)
    
    # Load results from knn, pre-edit metrics, and post-edit metrics
    restore_dir = os.path.join(trial_dir, 'models')
    knn_analysis_results = torch.load(os.path.join(restore_dir, knn_analysis_filename))
    pre_edit_metrics = torch.load(os.path.join(restore_dir, 'pre_edit_metrics.pth'))
    post_edit_metrics = torch.load(os.path.join(restore_dir, 'post_edit_metrics.pth'))
    
    # Combine results into one dictionary
    combined_results = combine_results(
        data_id=data_id,
        knn_analysis=knn_analysis_results,
        pre_edit_metrics=pre_edit_metrics,
        post_edit_metrics=post_edit_metrics)
    
    # Save column headers in first trial run
    if trial_idx == 0:
        column_headers = list(combined_results.keys())
    # Convert results to np.array & append to list
    combined_results = np.expand_dims(np.array(list(combined_results.values())), axis=0)
    data.append(combined_results)

# Convert data from list of np.arrays -> pd.DataFrame    
data = np.concatenate(data, axis=0)
df = pd.DataFrame(data, columns=column_headers)


6it [00:00, 375.43it/s]

saved/edit/trials/CINIC10_ImageNet-VGG_16/0110_120730/dog-train-n02114712_211/felzenszwalb_gaussian_0
saved/edit/trials/CINIC10_ImageNet-VGG_16/0110_120730/dog-train-n02114712_211/felzenszwalb_masked_0
saved/edit/trials/CINIC10_ImageNet-VGG_16/0110_120730/dog-train-n02114712_211/felzenszwalb_masked_1
saved/edit/trials/CINIC10_ImageNet-VGG_16/0110_120730/dog-train-n02114712_211/quickshift_masked_0
saved/edit/trials/CINIC10_ImageNet-VGG_16/0110_120730/dog-train-n02114712_211/slic_masked_0
saved/edit/trials/CINIC10_ImageNet-VGG_16/0110_120730/dog-train-n02114712_211/watershed_masked_1


In [11]:
# Save to CSV
csv_save_path = os.path.join(os.path.dirname(save_trials_path), 'results_table.csv')
df.to_csv(csv_save_path)